# ARC Reaction Demo using YAML files

Here we load our species objects from YAML files.

Note that no electronic structure job needs to be spawned. Of course, users can load only some of the species (or TSs) from YAML files and let ARC calculate the rest.

YAML files are generated automatically by <a href='http://reactionmechanismgenerator.github.io/RMG-Py/users/arkane/input.html#option-3-automatically-parse-yaml-files'>Arkane</a>, which means that ARC generates them as well. Look for them in the respective species / TS folder. These files compile the required QM information from all relevant jobs, so it's convinient to re-use the same species in a new ARC project w/o recalculating it (so saving computatinal resources). Also, this makes sharing species information much easier.

### imports

In [ ]:
import rmgpy
from rmgpy.molecule.molecule import *
from rmgpy.species import Species
from arc.species import ARCSpecies
from arc.reaction import ARCReaction
from IPython.display import display
import arc
import matplotlib.pyplot as plt
%matplotlib notebook

### Here we define ARCSpecies using YAML files

In [ ]:
# Define empty lists for convinience:
arc_species_list = list()
arc_rxn_list = list()

"""
Arguments for ARCSpecies:
------------------------
label (``str``)
is_ts (True/False, default is False)
rmg_species (an RMG `Species` object)
mol (an RMG `Molecule` object)
xyz (an xyz guess)
multiplicity (``int``, trivial value, 1 or 2, automatically determined)
charge (``int``, default: 0)
smiles (SMILES structure)
adjlist (RMG Adjacency List structure)
bond_corrections (BAC dictionary for Arkane)
generate_thermo (True/False, default is True)
yml_path (path to the YAML file)
ts_methods (a list with TS guess methods, default is in settings.py)

Note: A TS ARCSpecies object does not have to be created (unless the user would like to explicitly set arguments),
      it will be automatically created per ARCReaction object if not already defined.
"""

arc_species_list.append(ARCSpecies(label='ethanol', yml_path='yml/ethanol.yml'))
arc_species_list.append(ARCSpecies(label='OH', yml_path='yml/OH.yml'))
arc_species_list.append(ARCSpecies(label='o_ethanol_rad', yml_path='yml/o_ethanol_rad.yml'))
arc_species_list.append(ARCSpecies(label='H2O', yml_path='yml/H2O.yml'))

arc_species_list.append(ARCSpecies(label='TS', is_ts=True, yml_path='yml/ethanol + OH = o_ethanol_rad + H2O.yml'))

"""
Arguments for ARCReaction:
-------------------------
label (``str``) should be in the format 'reactant1 + reactant2(optional) <=> product1 + product2(optional)'
reactants (A list of labels corresponding to previously defined ARCSpecies objects)
products (A list of labels corresponding to previously defined ARCSpecies objects)
ts_label (A label corresponding to an ARCSpecies object)
rmg_reaction (An RMG `Reaction` object)
ts_methods (a list with TS guess methods, default is in settings.py, passed to the TS ARCSpecies object)
ts_xyz_guess (a list of xyz guesses)
multiplicity (trivial values automatically determined from reactants/products multiplicities)
charge (automatically determined from reactants)
"""

arc_rxn_list.append(ARCReaction(label='ethanol + OH <=> o_ethanol_rad + H2O', ts_label='TS', ts_methods=[]))

for spc in arc_species_list:
    if not spc.is_ts:
        display(spc.mol)

### Run ARC:

In [ ]:
"""
Arguments for ARC:
-----------------
project (``str``, the project name. Should be a valid name for a folder)
arc_species_list (A list of ARCSpecies and RMF `Species` objects)
arc_rxn_list (A list of ARCReacrion and RMF `Reaction` objects)
level_of_theory (e.g., `CCSD(T)-F12a/aug-cc-pVTZ//B3LYP/6-311++G(3df,3pd)` or `CBS-QB3`...)
conformer_level
composite_method (either pass the composite method here, or in `level_of_theory`)
opt_level
freq_level (set to `opt_level` if not specified)
sp_level
scan_level
ts_guess_level
fine (True/False, default is True)
generate_conformers (True/False, default is True)
scan_rotors (True/False, default is True)
use_bac (True/False for using bond additivity corrections, default is True)
model_chemistry (should be a valid Arkane "model chemistry")
ess_settings (a software:server dictionary, overrides ARC's servers scanner)
initial_trsh (an initial troubleshooting method for jobs)
t_min (float, minimal T for kineics computations, default: 300)
t_max (float, maximal T for kineics computations, default: 3000)
t_count (int The number of temperature points between t_min and t_max for kinetics computations)
verbose (logging verbosity)
project_directory (A path to the project directory, defaults to ARC/Projects/<project name>)
"""

arc0 = arc.ARC(project='rxn_tst13', arc_species_list=arc_species_list,
              arc_rxn_list=arc_rxn_list, scan_rotors=False, fine=False)
arc0.execute()